In [2]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [10]:
path = 'C:/Users/kt NexR/PycharmProjects/Recommendation-algorithms/Classic_recommendation_algorithms/contents-based/data'
rating_df = pd.read_csv(path+'/ratings.csv', encoding='utf-8')
movies_df = pd.read_csv(path+'/movies.csv', index_col='movieId', encoding='utf-8')
tags_df = pd.read_csv(path+'/tags.csv', encoding='utf-8')

In [11]:
movies_df.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [13]:
rating_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [12]:
tags_df.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [17]:
total_count = len(movies_df.index)
total_genres = list(set([genre for sublist in list(map(lambda x: x.split('|'), movies_df['genres'])) for genre in sublist]))
total_genres

['Western',
 'Film-Noir',
 'Children',
 'Thriller',
 'Comedy',
 'Horror',
 'Adventure',
 'Action',
 'Drama',
 '(no genres listed)',
 'Animation',
 'Mystery',
 'Documentary',
 'Romance',
 'IMAX',
 'Fantasy',
 'Sci-Fi',
 'Musical',
 'War',
 'Crime']

In [18]:
print(f"전체 영화 수: {total_count}")
print(f"장르: {total_genres}")

전체 영화 수: 9742
장르: ['Western', 'Film-Noir', 'Children', 'Thriller', 'Comedy', 'Horror', 'Adventure', 'Action', 'Drama', '(no genres listed)', 'Animation', 'Mystery', 'Documentary', 'Romance', 'IMAX', 'Fantasy', 'Sci-Fi', 'Musical', 'War', 'Crime']


In [19]:
genre_count = dict.fromkeys(total_genres)

for each_genre_list in movies_df['genres']:
    for genre in each_genre_list.split('|'):
        if genre_count[genre] == None:
            genre_count[genre] = 1
        else :
            genre_count[genre] = genre_count[genre] +1

In [20]:
genre_count

{'Western': 167,
 'Film-Noir': 87,
 'Children': 664,
 'Thriller': 1894,
 'Comedy': 3756,
 'Horror': 978,
 'Adventure': 1263,
 'Action': 1828,
 'Drama': 4361,
 '(no genres listed)': 34,
 'Animation': 611,
 'Mystery': 573,
 'Documentary': 440,
 'Romance': 1596,
 'IMAX': 158,
 'Fantasy': 779,
 'Sci-Fi': 980,
 'Musical': 334,
 'War': 382,
 'Crime': 1199}

In [21]:
for each_genre in genre_count:
    genre_count[each_genre] = np.log10(total_count/genre_count[each_genre]) #idf
genre_count #idf??

{'Western': 1.7659316540881678,
 'Film-Noir': 2.0491288726171324,
 'Children': 1.1664800458677336,
 'Thriller': 0.7112681505684965,
 'Comedy': 0.4139225416416778,
 'Horror': 0.9983092704481497,
 'Adventure': 0.8872447746804204,
 'Action': 0.7266719338379385,
 'Drama': 0.3490620385623247,
 '(no genres listed)': 2.457169208193496,
 'Animation': 1.2026069149931968,
 'Mystery': 1.2304935032683613,
 'Documentary': 1.3451954487495636,
 'Romance': 0.7856152382210405,
 'IMAX': 1.7899910382813284,
 'Fantasy': 1.0971106675631865,
 'Sci-Fi': 0.9974220495432563,
 'Musical': 1.4649016584241867,
 'War': 1.4065847623240424,
 'Crime': 0.9098289421369025}

In [22]:
genre_representation = pd.DataFrame(columns=sorted(total_genres), index=movies_df.index)
genre_representation.head()

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
for index, each_row in tqdm(movies_df.iterrows()):
    dict_temp = {i: genre_count[i] for i in each_row['genres'].split('|')}
    row_to_add = pd.DataFrame(dict_temp, index=[index])
    genre_representation.update(row_to_add)

genre_representation


0it [00:00, ?it/s]
7it [00:00, 66.03it/s]
17it [00:00, 82.27it/s]
26it [00:00, 75.51it/s]
34it [00:00, 76.29it/s]
43it [00:00, 79.60it/s]
52it [00:00, 73.86it/s]
60it [00:00, 71.08it/s]
69it [00:00, 74.75it/s]
77it [00:01, 75.07it/s]
87it [00:01, 80.08it/s]
96it [00:01, 80.94it/s]
105it [00:01, 77.84it/s]
114it [00:01, 81.07it/s]
124it [00:01, 84.67it/s]
133it [00:01, 82.89it/s]
142it [00:01, 73.62it/s]
150it [00:01, 72.78it/s]
158it [00:02, 70.81it/s]
166it [00:02, 72.24it/s]
174it [00:02, 71.88it/s]
182it [00:02, 71.41it/s]
192it [00:02, 76.86it/s]
201it [00:02, 79.41it/s]
209it [00:02, 71.82it/s]
217it [00:02, 72.79it/s]
225it [00:02, 70.80it/s]
233it [00:03, 68.04it/s]
240it [00:03, 67.45it/s]
250it [00:03, 74.66it/s]
259it [00:03, 77.18it/s]
269it [00:03, 81.86it/s]
278it [00:03, 82.55it/s]
287it [00:03, 76.55it/s]
295it [00:03, 71.01it/s]
303it [00:04, 71.92it/s]
311it [00:04, 67.47it/s]
319it [00:04, 69.38it/s]
327it [00:04, 71.34it/s]
335it [00:04, 72.16it/s]
343it [00:04, 66.

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0.887245,1.202607,1.16648,0.413923,NaN,NaN,NaN,1.097111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,0.887245,NaN,1.16648,NaN,NaN,NaN,NaN,1.097111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,0.413923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.785615,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,0.413923,NaN,NaN,0.349062,NaN,NaN,NaN,NaN,NaN,NaN,0.785615,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,0.413923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,NaN,0.726672,NaN,1.202607,NaN,0.413923,NaN,NaN,NaN,1.097111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193583,NaN,NaN,NaN,1.202607,NaN,0.413923,NaN,NaN,NaN,1.097111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.349062,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
tags_df.head(30)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
5,2,89774,Tom Hardy,1445715205
6,2,106782,drugs,1445715054
7,2,106782,Leonardo DiCaprio,1445715051
8,2,106782,Martin Scorsese,1445715056
9,7,48516,way too long,1169687325


In [34]:
movies_df.loc[89774]

title     Warrior (2011)
genres             Drama
Name: 89774, dtype: object

In [35]:
tag_column = list(map(lambda x: x.split(','), tags_df['tag']))
unique_tags = list(set(list(map(lambda x: x.strip(), list([tag for sublist in tag_column for tag in sublist])))))

In [40]:
# Compute IDF for tag
total_movie_count = len(set(tags_df['movieId']))
# key: tag, value: number of movies with such tag
tag_count_dict = dict.fromkeys(unique_tags)

for each_movie_tag_list in tags_df['tag']:
    for tag in each_movie_tag_list.split(","):
        if tag_count_dict[tag.strip()] == None:
            tag_count_dict[tag.strip()] = 1
        else:
            tag_count_dict[tag.strip()] += 1

tag_idf = dict()
for each_tag in tag_count_dict:
    tag_idf[each_tag] = np.log10(total_movie_count / tag_count_dict[each_tag])

tag_idf

{'Magic': 2.4183012913197452,
 'new york': 2.895422546039408,
 'dc comics': 2.895422546039408,
 'truth': 3.196452541703389,
 'Scott Turow': 3.196452541703389,
 'bad science': 3.196452541703389,
 'italy': 3.196452541703389,
 'Mrs. DeWinter': 3.196452541703389,
 'money': 3.196452541703389,
 'food': 2.7193312869837265,
 'martial arts': 2.2933625547114453,
 'noir': 3.196452541703389,
 'television': 2.4183012913197452,
 'avant-garde romantic comedy': 3.196452541703389,
 'entertaining': 3.196452541703389,
 'coen brothers': 3.196452541703389,
 'ghosts': 2.196452541703389,
 'Dr. Strange': 3.196452541703389,
 'Witty': 3.196452541703389,
 'Shakespeare': 2.1172712956557644,
 'Ryan Reynolds': 2.5943925503754266,
 'aging': 3.196452541703389,
 'video game adaptation': 3.196452541703389,
 'stupid is as stupid does': 3.196452541703389,
 'Romans': 3.196452541703389,
 'Mental Hospital': 3.196452541703389,
 'Prince': 3.196452541703389,
 'Judaism': 2.4974825373673704,
 'art house': 3.196452541703389,
 'la

In [41]:
len(tag_idf.keys())

1589